In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

print("Import OK")

Using TensorFlow backend.


Import OK


In [7]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024, activation='relu')(x) #dense layer 2
x = Dense(512, activation='relu')(x) #dense layer 3
#x = Dense(256, activation='relu')(x) #dense layer 4
x = Dense(128, activation='relu')(x) #dense layer 5
#x = Dense(64, activation='relu')(x) #dense layer 6
#x = Dropout(0.3)(x)
preds = Dense(13, activation='softmax')(x) #final layer with softmax activation

model2 = Model(inputs=model.input, outputs=preds)
for layer in model2.layers[:20]:
    layer.trainable=False
for layer in model2.layers[20:]:
    layer.trainable=True
    
model2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [8]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
                                                 target_size=(256, 256),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2819 images belonging to 13 classes.


In [9]:
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model2.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=10)

Epoch 1/10
88/88 [==============================] - 10s 109ms/step - loss: 0.7352 - accuracy: 0.7804
Epoch 2/10
88/88 [==============================] - 9s 107ms/step - loss: 0.2246 - accuracy: 0.9290
Epoch 3/10
88/88 [==============================] - 9s 105ms/step - loss: 0.1241 - accuracy: 0.9577
Epoch 4/10
88/88 [==============================] - 9s 105ms/step - loss: 0.0919 - accuracy: 0.9713
Epoch 5/10
88/88 [==============================] - 9s 106ms/step - loss: 0.0548 - accuracy: 0.9821
Epoch 6/10
88/88 [==============================] - 10s 110ms/step - loss: 0.0375 - accuracy: 0.9883
Epoch 7/10
88/88 [==============================] - 9s 108ms/step - loss: 0.0203 - accuracy: 0.9964
Epoch 8/10
88/88 [==============================] - 9s 105ms/step - loss: 0.0114 - accuracy: 0.9982
Epoch 9/10
88/88 [==============================] - 9s 106ms/step - loss: 0.0109 - accuracy: 0.9972
Epoch 10/10
88/88 [==============================] - 10s 108ms/step - loss: 0.0087 - accuracy: 0.9

In [10]:
import cv2

test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [11]:
def processing(image_data):
    #img = [data[123], data[1054], data[156]]
    #print("Original size:", img[1].shape)
    #print(training_label[1054])
    #original = img[1]
    #display_one(original)
    
    height = 256
    width = 256
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

print(test_data_reshape[0].shape)

(256, 256, 3)


In [12]:
#for i in range(len(test_data_reshape)):
#    test_data_reshape[i] = (test_data_reshape[i] - 128.0) / 128.0
    
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 256, 256, 3)
    
scene_detail = ["bedroom", "coast", "forest", "highway", "insidecity", "kitchen", "livingroom", "mountain", "office", "opencountry", "street", "suburb", "tallbuilding"]

print("id,label")

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    answer = model2.predict(test_data_reshape[i])
    print(name+","+scene_detail[np.argmax(answer)])


id,label
image_0545,bedroom
image_0565,tallbuilding
image_0897,opencountry
image_0007,forest
image_0438,street
image_0741,livingroom
image_1037,coast
image_0706,opencountry
image_0698,street
image_0928,mountain
image_0270,kitchen
image_0472,street
image_0019,insidecity
image_0442,livingroom
image_0247,mountain
image_0673,suburb
image_0984,mountain
image_0208,suburb
image_0364,forest
image_0508,mountain
image_1026,suburb
image_0463,opencountry
image_0826,suburb
image_0138,insidecity
image_0268,mountain
image_0832,suburb
image_0248,suburb
image_0885,suburb
image_0140,bedroom
image_0622,insidecity
image_0639,street
image_0117,office
image_0910,office
image_0127,street
image_0621,insidecity
image_0607,kitchen
image_0401,kitchen
image_0099,bedroom
image_0682,kitchen
image_0665,coast
image_0733,bedroom
image_0229,suburb
image_0726,forest
image_0045,street
image_0232,forest
image_0869,forest
image_0415,street
image_0167,bedroom
image_0223,street
image_1009,tallbuilding
image_0372,livingroom
i

image_0802,tallbuilding
image_0195,insidecity
image_0109,office
image_0080,mountain
image_0196,livingroom
image_0115,street
image_1007,highway
image_0305,suburb
image_0782,street
image_0164,coast
image_0940,street
image_0325,tallbuilding
image_0539,highway
image_0573,insidecity
image_0448,coast
image_0801,mountain
image_0661,livingroom
image_0969,suburb
image_0583,bedroom
image_0234,street
image_0943,bedroom
image_0563,highway
image_0695,coast
image_1027,mountain
image_0606,insidecity
image_0328,bedroom
image_0558,street
image_0572,tallbuilding
image_1032,insidecity
image_0589,coast
image_0748,opencountry
image_0900,bedroom
image_0071,highway
image_0191,mountain
image_0425,livingroom
image_0666,office
image_0361,kitchen
image_0959,mountain
image_0601,office
image_0432,coast
image_0345,livingroom
image_1036,opencountry
image_0486,insidecity
image_0596,highway
image_0504,highway
image_0314,suburb
image_0735,livingroom
image_0376,kitchen
image_0239,coast
image_0692,forest
image_0399,livin

image_0211,kitchen
image_0852,mountain
image_0313,forest
image_0815,insidecity
image_0011,livingroom
image_0664,forest
image_0039,suburb
image_0935,opencountry
image_0737,insidecity
image_0848,kitchen
image_0424,coast
image_0514,tallbuilding
image_0125,coast
image_0383,bedroom
image_0452,highway
image_0355,tallbuilding
image_0435,coast
image_0315,suburb
image_0101,coast
image_0546,mountain
image_0254,livingroom
image_0224,opencountry
image_0756,suburb
image_0430,forest
image_0296,tallbuilding
image_0633,opencountry
image_1002,forest
image_0879,opencountry
image_0179,insidecity
image_0553,tallbuilding
image_0915,forest
image_0136,tallbuilding
image_0385,highway
image_0029,forest
image_0060,forest
image_0672,insidecity
image_0976,suburb
image_0076,insidecity
image_0818,tallbuilding
image_0501,coast
image_0883,bedroom
image_0663,livingroom
image_0289,kitchen
image_0156,tallbuilding
image_0320,highway
image_0613,office
image_0370,kitchen
image_0497,livingroom
image_0820,mountain
image_0205